Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import re

from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score

In [6]:
RANDOM_STATE = 12345

In [7]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Добавим колонку `text_lemmatized`. В итоге в ней окажется лемматизированный текст.

In [8]:
data['text_lemmatized'] = data['text'].values.astype('U')

# Сначала оставим в каждом твите только буквы и апостроф "'", чтобы сохранить I'm и т.п.
data['text_lemmatized'] = data['text_lemmatized'].apply(lambda x: re.sub(r'[^a-zA-Z\']', ' ', x))

# Затем переведем весь текст в нижний регистр
data['text_lemmatized'] = data['text_lemmatized'].str.lower()
data['text_lemmatized'] = data['text_lemmatized'].apply(lambda x: ' '.join(x.split()))

In [29]:
print('BEFORE')
print(data.loc[0, 'text'])
print('AFTER')
print(data.loc[0, 'text_lemmatized'])

BEFORE
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
AFTER
explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now


Дополнительно **проведем лемматизацию**. Использую готовый пример лемматизации **с использованием WordNetLemmatizer**.

In [30]:
%%time

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# 2. Lemmatize a Sentence with the appropriate POS tag
data['text_lemmatized'] = data['text_lemmatized'].apply(
    lambda x: ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)]))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Wall time: 1h 9min 51s


In [31]:
print('BEFORE')
print(data.loc[0, 'text'])
print('AFTER')
print(data.loc[0, 'text_lemmatized'])

BEFORE
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
AFTER
explanation why the edits make under my username hardcore metallica fan be revert they be n't vandalism just closure on some gas after i vote at new york doll fac and please do n't remove the template from the talk page since i 'm retire now


# 2. Обучение

In [32]:
# Разделим данные на 2 выборки
train, test = train_test_split(data, test_size=0.25, random_state=RANDOM_STATE)
print(train.shape)
print(test.shape)

(119678, 3)
(39893, 3)


## Мешок слов

Для начала попробуем применить простой мешок слов.

In [36]:
count_tfidf = CountVectorizer()
tfidf_train = count_tfidf.fit_transform(train['text_lemmatized'])
tfidf_test = count_tfidf.transform(test['text_lemmatized'])

model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(tfidf_train, train['toxic'])
pred = model.predict(tfidf_test)
f1_score1 = f1_score(test['toxic'], pred)

In [37]:
print(f1_score1)

0.7249010699105967


Результат неплохой, но пока не 0,75.

## TD-IDF

Сравним кодирование с TD-IDF со следующими параметрами:
1. без использования/с использованием стоп-слов
2. 1/2/3-грамы

In [33]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
%%time

params_grid = {
    'ngram_range': [(1,1), (2,2), (3,3)],
    'stop_words': [None, stop_words_set]
}

# Результаты на каждой итерации будем сохранять в переменные
print('|{:<20}|{:<20}|{:<20}|{:<20}|'.format('ngrams', 'Стоп-слова', 'Признаков', 'f1_score'))

for ngram_range in params_grid['ngram_range']:
    for stop_words in params_grid['stop_words']:
        
        count_tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=ngram_range)
        tfidf_train = count_tfidf.fit_transform(train['text_lemmatized'])
        tfidf_test = count_tfidf.transform(test['text_lemmatized'])
        
        model = LogisticRegression(random_state=RANDOM_STATE)
        model.fit(tfidf_train, train['toxic'])
        pred = model.predict(tfidf_test)
        f1_score1 = f1_score(test['toxic'], pred)
        
        print('|{:^20}|{:<20}|{:>20}|{:>20.6f}|'.format(ngram_range[0], f'{"<не выбраны>" if stop_words == None else "<выбраны>"}', tfidf_train.shape[1], f1_score1))

|ngrams              |Стоп-слова          |Признаков           |f1_score            |
|         1          |<не выбраны>        |              128700|            0.756341|
|         1          |<выбраны>           |              128560|            0.743291|
|         2          |<не выбраны>        |             1535661|            0.486866|
|         2          |<выбраны>           |             1862025|            0.227723|
|         3          |<не выбраны>        |             4043404|            0.156880|
|         3          |<выбраны>           |             3171497|            0.016019|
Wall time: 2min 47s


# 3. Выводы

Мы достигли заданной величины метрики F1. 

**Лучший результат** был получен при использовании **униграм без использования словаря стоп-слов** (0,756341).

Интересно, что качество модели **ухудшается при росте количества грам**.